In [1]:
import pandas as pd

In [2]:
def clean_color(color):
    try:
        if color[:2] in ["WH", "WT"]:
            return "WHITE"
        elif color in ["BLACK", "BK", "BL", "BLK",]:
            return "BLACK"
        elif color in ["GREY", "GRAY", "GY", "GR", "GRY"]:
            return "GRAY"
        elif color[:2] in ["SI", "SL"]:
            return "GRAY"
        elif color in ["BLUE", "BLU"]:
            return "BLUE"
        elif color in ["RD", "RED"]:
            return "RED"
        elif color in ["GREEN", "GN", "GRN"]:
            return "GREEN"
        elif color in ["YW", "YELLO", "YELLW", "GL", "GOLD"]:
            return "YELLOW"
        elif color in ["BROWN", "BR", "TN", "TAN", "BRN"]:
            return "BROWN"
        else:
            return "OTHER"
    except:
        return "OTHER"
    
# def clean_date(date, year):
#     try:
#         if int(date[-4:]) == year:
#             if int(date[:2]) > 6:
#                 return None
#             else:
#                 return date
#         else:
#             if int(date[:2]) <= 6:
#                 return None
#             else:
#                 return date
#     except:
#         return None

In [3]:
year_list = [i for i in range(2014, 2024)]
flag = 0

for year in year_list:

    print(f"Started {year}")
    
    # load data
    data = pd.read_csv(f"../../data/main_data/color_and_plate_type/Parking_Violations_Issued_{str(year)}.csv",
                       dtype="string")
    
    # create month and year columns, convert precinct to int
    data["Month"] = data["Issue Date"].map(lambda x: int(x[:2]))
    data["Year"] = data["Issue Date"].map(lambda x: int(x[-4:]))
    data["Violation Precinct"] = data["Violation Precinct"].map(lambda x: int(x))
    
    # drop unnecessary columns
    data.drop(columns=["Issue Date", "Plate Type"], inplace=True)

    # clean colors and dates
    data["Vehicle Color"] = data["Vehicle Color"].map(lambda x: clean_color(x))

    # complicated aggregate queries
    # STEP 1: group by and get the color with max frequency in each time period and precinct
    data2 = data.groupby(by = ["Year","Month","Violation Precinct","Vehicle Color"], 
                        as_index=False).size()
    data2 = data2.sort_values(by = ["Year","Month","Violation Precinct","size"],
                            ascending = [True, True, True, False] )
    data2 = data2.drop_duplicates(["Year","Month","Violation Precinct"])
    data2.reset_index(drop=True, inplace=True)

    # STEP 2: group by and get the total violations in each time period and precinct
    data_max = data.groupby(by = ["Year","Month","Violation Precinct",], 
                     as_index=False).size()
    data_max = data_max.sort_values(by = ["Year","Month","Violation Precinct",],)
    
    # STEP 3: compute percentage, rename columns
    data4 = data2
    data4["Total"] = data_max["size"]
    data4["Percentage"] = 100*data4["size"]/data4["Total"]
    data4.rename(columns={
                    "size": "total_maxcolor",
                    "Total": 'total_allcolors',
                    "Percentage": "percent_maxcolor",
                }, inplace=True)
    
    # append this year's data to final_data
    if not flag:
        flag = 1
        final_data = data4 
    else:
        final_data = pd.concat([final_data, data4])
    
    print(f"Finished {str(year)}")

final_data.to_csv(f"../../data/final_data_to_join/color_data.csv", index=False)

Started 2014
Finished 2014
Started 2015
Finished 2015
Started 2016
Finished 2016
Started 2017
Finished 2017
Started 2018
Finished 2018
Started 2019
Finished 2019
Started 2020
Finished 2020
Started 2021
Finished 2021
Started 2022
Finished 2022
Started 2023
Finished 2023


In [4]:
# data2 = data.groupby(by = ["Year","Month","Violation Precinct","Vehicle Color"], 
#                      as_index=False).size()
# data2 = data2.sort_values(by = ["Year","Month","Violation Precinct","size"],
#                           ascending = [True, True, True, False] )
# data2 = data2.drop_duplicates(["Year","Month","Violation Precinct"])
# # data2.to_csv(f"../Data/final_data_to_join/temp.csv", index=False)
# data2.reset_index(drop=True, inplace=True)
# data2

In [5]:
# data_max = data.groupby(by = ["Year","Month","Violation Precinct",], 
#                      as_index=False).size()
# data_max = data_max.sort_values(by = ["Year","Month","Violation Precinct",],)
# data_max

In [6]:
# data4 = data2
# data4["Total"] = data_max["size"]
# data4["Percentage"] = 100*data4["size"]/data4["Total"]
# data4.rename(columns={
#                 "size": "total_maxcolor",
#                 "Total": 'total_allcolors',
#                 "Percentage": "percent_maxcolor",
#             }, inplace=True)
# data4.to_csv(f"../Data/final_data_to_join/color_data.csv", index=False)

In [7]:
# vc_color = data["Vehicle Color"].value_counts()
# l1 = vc_color.index.tolist()
# l2 = vc_color.tolist()
# l = [[l1[i], l2[i]] for i in range(len(vc_color))]
# l

In [8]:
# data2 = data["Vehicle Color"].map(lambda x: clean_color(x))
# vc_color = data2.value_counts()
# l1 = vc_color.index.tolist()
# l2 = vc_color.tolist()
# l = [[l1[i], l2[i]] for i in range(len(vc_color))]
# l

In [9]:
# data[["Month", "Year"]].value_counts()

In [10]:
# data2 = data["Violation Precinct"]
# vc_color = data2.value_counts()
# l1 = vc_color.index.tolist()
# l2 = vc_color.tolist()
# l = [[l1[i], l2[i]] for i in range(len(vc_color))]
# l